In [1]:
import numpy as np 
import tensorflow as tf
from tensorflow import keras

<h1>Download, Prepare, and Configure Data </h1>

In [2]:
# Path to the data txt file on disk.
data_path = "fra.txt"

In [3]:
data_path

'fra.txt'

In [4]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.

In [5]:
#vectorize data and sort characters sets from texts
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

i=0
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
"""
lines first 2 line values below

input_text: Go.
target_text: Va !
_: CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)

input_text: Go.
target_text: Marche.
_: CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8090732 (Micsmithel)

"""
#only go to number 9999 (10000-1)
for line in lines[:min(num_samples,len(lines)-1)]:
    i+=1
    if i<250:
        input_text, target_text, _ = line.split("\t")
        # tab will denote start sequence and new line char will denote end sequence
        target_text = "\t"+ target_text + "\n"
        input_texts.append(input_text)
        target_texts.append(target_text)
        for char in input_text:
            input_characters.add(char)
        for char in target_text:
            target_characters.add(char)
            

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
# max length of input text (English word, phrase, sentence, etc.)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
# max length of target text (French word, phrase, sentence, etc.)
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [6]:
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 249
Number of unique input tokens: 47
Number of unique output tokens: 71
Max sequence length for inputs: 8
Max sequence length for outputs: 32


In [10]:
# Index input and output tokens
input_token_index = dict([(char,i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char,i) for i, char in enumerate(target_characters)])

In [11]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")

In [28]:
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

In [29]:
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

In [41]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

<h1>Build the model</h1>

In [42]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))

In [44]:
encoder = keras.layers.LSTM(latent_dim, return_state=True)

In [46]:
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

In [50]:
# Discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

In [52]:
# Set up our decoder to return full output sequences,
# Return internal states
# Return states not used in the training model, but used in inference
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

In [54]:
# Define the model that will take `encoder_input_data` & `decoder_input_data`
# and return `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

<h1>Train the model</h1>

In [ ]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])